In [1]:
# Library Imports.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Allows plots to appear directly in the notebook.
%matplotlib inline

from patsy import dmatrices
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score

In [2]:
df = pd.read_csv('full_dataframe.csv', index_col=0)
df.dtypes

number                int64
name                 object
status               object
bike_stands           int64
available_bikes       int64
available_stands      int64
last_update          object
banking               int64
weather_id            int64
main                 object
description          object
temperature         float64
humidity            float64
wind_speed          float64
dtype: object

In [3]:
df.dtypes

number                int64
name                 object
status               object
bike_stands           int64
available_bikes       int64
available_stands      int64
last_update          object
banking               int64
weather_id            int64
main                 object
description          object
temperature         float64
humidity            float64
wind_speed          float64
dtype: object

In [4]:
df['description'].value_counts()

broken clouds                   254111
light rain                      173894
scattered clouds                109585
few clouds                      103236
clear sky                        73475
moderate rain                    63231
shower rain                      55117
light intensity shower rain      37147
mist                             28452
heavy intensity rain             18499
light intensity drizzle          16280
fog                              14763
light intensity drizzle rain      4232
very heavy rain                   3052
snow                              1458
shower sleet                      1140
overcast clouds                    240
Name: description, dtype: int64

In [5]:
df['main'].value_counts()

Clouds     467172
Rain       350940
Clear       73475
Mist        28452
Drizzle     20512
Fog         14763
Snow         2598
Name: main, dtype: int64

In [6]:
for col in df[['description']].columns:
    if [df['description']=='clear sky']:
        df['description'].replace('clear sky',1, inplace=True)
    if[df['description']=='few clouds']:
        df['description'].replace('few clouds', 2, inplace=True)
    if[df['description']=='scattered clouds']:
        df['description'].replace('scattered clouds',3, inplace=True)
    if [df['description']=='broken clouds']:
        df['description'].replace('broken clouds',4, inplace=True)
    if [df['description']=='fog']:
        df['description'].replace('fog',5, inplace=True)
    if [df['description']=='mist']:
        df['description'].replace('mist',6, inplace=True)
    if [df['description']=='light intensity drizzle']:
        df['description'].replace('light intensity drizzle',7, inplace=True)
    if [df['description']=='light intensity drizzle rain']:
        df['description'].replace('light intensity drizzle rain',8, inplace=True)
    if [df['description']=='light intensity shower rain']:
        df['description'].replace('light intensity shower rain',9, inplace=True)
    if [df['description']=='shower rain']:
        df['description'].replace('shower rain',10, inplace=True)
    if [df['description']=='heavy intensity rain']:
        df['description'].replace('heavy intensity rain',11, inplace=True)
    if [df['description']=='light rain']:
        df['description'].replace('light rain',12, inplace=True)
    if [df['description']=='moderate rain']:
        df['description'].replace('moderate rain',13, inplace=True)
    if [df['description']=='very heavy rain']:
        df['description'].replace('very heavy rain',14, inplace=True)
    if [df['description']=='shower sleet']:
        df['description'].replace('shower sleet',15, inplace=True)
    if [df['description']=='snow']:
        df['description'].replace('snow',16, inplace=True)

In [7]:
df['description'].value_counts()

4                  254111
12                 173894
3                  109585
2                  103236
1                   73475
13                  63231
10                  55117
9                   37147
6                   28452
11                  18499
7                   16280
5                   14763
8                    4232
14                   3052
16                   1458
15                   1140
overcast clouds       240
Name: description, dtype: int64

In [8]:
df['last_update'] = df['last_update'].astype('datetime64[ns]')

In [9]:
df['week'] = df['last_update'].dt.week

In [10]:
df['day'] = df['last_update'].dt.weekday

In [11]:
df.dtypes

number                       int64
name                        object
status                      object
bike_stands                  int64
available_bikes              int64
available_stands             int64
last_update         datetime64[ns]
banking                      int64
weather_id                   int64
main                        object
description                 object
temperature                float64
humidity                   float64
wind_speed                 float64
week                         int64
day                          int64
dtype: object

In [12]:
df['last_update'] = df['last_update'].dt.round('30min') 

In [13]:
df['time'] =df.last_update.map(lambda t: t.strftime('%H%M'))

In [14]:
df = df.drop_duplicates()

In [15]:
df = df.drop(['last_update', 'number', 'banking', 'bike_stands', 'weather_id', 'main', 'description', 'temperature', 'humidity', 'wind_speed'] , axis=1)

In [16]:
df = df.loc[(df['day'] == 4) & (df['name'] == 'GRAND CANAL DOCK')]

In [17]:
df.reset_index(drop=True)

,name,status,available_bikes,available_stands,week,day,time
0,GRAND CANAL DOCK,OPEN,10,30,8,4,0000
1,GRAND CANAL DOCK,OPEN,10,30,8,4,0030
2,GRAND CANAL DOCK,OPEN,9,31,8,4,0030
3,GRAND CANAL DOCK,OPEN,9,31,8,4,0100
4,GRAND CANAL DOCK,OPEN,9,31,8,4,0130
5,GRAND CANAL DOCK,OPEN,9,31,8,4,0200
6,GRAND CANAL DOCK,OPEN,9,31,8,4,0230
7,GRAND CANAL DOCK,OPEN,9,31,8,4,0300
8,GRAND CANAL DOCK,OPEN,9,31,8,4,0330
9,GRAND CANAL DOCK,OPEN,9,31,8,4,0400


In [18]:
df.dtypes

name                object
status              object
available_bikes      int64
available_stands     int64
week                 int64
day                  int64
time                object
dtype: object

In [19]:
# df.groupby('time')['available_bikes'].mean()

In [20]:
df['Mean_Bikes'] = df.groupby('time')['available_bikes'].transform('mean').astype(int)

In [21]:
df.dtypes

name                object
status              object
available_bikes      int64
available_stands     int64
week                 int64
day                  int64
time                object
Mean_Bikes           int64
dtype: object

In [22]:
df

,name,status,available_bikes,available_stands,week,day,time,Mean_Bikes
12772,GRAND CANAL DOCK,OPEN,10,30,8,4,0000,5
13056,GRAND CANAL DOCK,OPEN,10,30,8,4,0030,3
13254,GRAND CANAL DOCK,OPEN,9,31,8,4,0030,3
13496,GRAND CANAL DOCK,OPEN,9,31,8,4,0100,3
13840,GRAND CANAL DOCK,OPEN,9,31,8,4,0130,3
14181,GRAND CANAL DOCK,OPEN,9,31,8,4,0200,3
14522,GRAND CANAL DOCK,OPEN,9,31,8,4,0230,3
14910,GRAND CANAL DOCK,OPEN,9,31,8,4,0300,3
15318,GRAND CANAL DOCK,OPEN,9,31,8,4,0330,3
15677,GRAND CANAL DOCK,OPEN,9,31,8,4,0400,3


In [23]:
df.drop_duplicates(subset=['time'], keep='first')

,name,status,available_bikes,available_stands,week,day,time,Mean_Bikes
12772,GRAND CANAL DOCK,OPEN,10,30,8,4,0000,5
13056,GRAND CANAL DOCK,OPEN,10,30,8,4,0030,3
13496,GRAND CANAL DOCK,OPEN,9,31,8,4,0100,3
13840,GRAND CANAL DOCK,OPEN,9,31,8,4,0130,3
14181,GRAND CANAL DOCK,OPEN,9,31,8,4,0200,3
14522,GRAND CANAL DOCK,OPEN,9,31,8,4,0230,3
14910,GRAND CANAL DOCK,OPEN,9,31,8,4,0300,3
15318,GRAND CANAL DOCK,OPEN,9,31,8,4,0330,3
15677,GRAND CANAL DOCK,OPEN,9,31,8,4,0400,3
16067,GRAND CANAL DOCK,OPEN,9,31,8,4,0430,3


In [25]:
df = df.sort_values(by=['Mean_Bikes'])

In [26]:
df

,name,status,available_bikes,available_stands,week,day,time,Mean_Bikes
609481,GRAND CANAL DOCK,OPEN,1,39,11,4,0330,3
384612,GRAND CANAL DOCK,OPEN,2,38,10,4,0230,3
384879,GRAND CANAL DOCK,OPEN,2,38,10,4,0300,3
385146,GRAND CANAL DOCK,OPEN,2,38,10,4,0330,3
385421,GRAND CANAL DOCK,OPEN,2,38,10,4,0330,3
385654,GRAND CANAL DOCK,OPEN,2,38,10,4,0400,3
385796,GRAND CANAL DOCK,OPEN,2,38,10,4,0400,3
386054,GRAND CANAL DOCK,OPEN,2,38,10,4,0430,3
386300,GRAND CANAL DOCK,OPEN,2,38,10,4,0430,3
607078,GRAND CANAL DOCK,OPEN,0,40,11,4,0030,3


## Preparing the columns

In [30]:
item_list = list(range(df.shape[1]))
list_to_remove = [0,1,2,7]
final_list= list(set(item_list).difference(set(list_to_remove)))
print(final_list)

[3, 4, 5, 6]


In [31]:
X = df.iloc[:, final_list].values
y = df.iloc[:, 7].values

## Splitting to train and test set & Creating Model

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25) 

In [33]:
sc = StandardScaler()  
X_train = sc.fit_transform(X_train)  
X_test = sc.transform(X_test)

/Users/Stephen/anaconda3/envs/dublinbikes/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/Stephen/anaconda3/envs/dublinbikes/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/Stephen/anaconda3/envs/dublinbikes/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [34]:
# Running the X_train and y_train on the RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=100, random_state=0)  
regressor.fit(X_train, y_train)  

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

## Testing the model on X_train

In [35]:
y_pred = regressor.predict(X_train)  

In [36]:
print('MAE:', metrics.mean_absolute_error(y_train, y_pred))  
print('MSE:', metrics.mean_squared_error(y_train, y_pred))  
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_train, y_pred)))
print('R2: ', metrics.r2_score(y_train,y_pred))

MAE: 0.028719999999999964
MSE: 0.012813599999999994
RMSE: 0.113197173109579
R2:  0.9999064996921158


## Testing the model on X_test

In [37]:
y_pred = regressor.predict(X_test)  

In [38]:
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))  
print('MSE:', metrics.mean_squared_error(y_test, y_pred))  
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2: ', metrics.r2_score(y_test,y_pred))

MAE: 0.15359281437125738
MSE: 0.7704508982035928
RMSE: 0.8777533242338605
R2:  0.9944474765949892


## Cross-validation testing

In [ ]:
scoring = {'abs_error': 'neg_mean_absolute_error', 'squared_error': 'neg_mean_squared_error'}

scores = cross_val_score(regressor, X_test, y_test, cv=10, scoring='neg_mean_absolute_error')

scores
# predictions = cross_val_predict(scores, X, y, cv=10)

In [ ]:
# Looking at the co-efficient importance for each feature
print("Features and co-efficients: " , list(zip(['week', 'day' , 'time'], regressor.feature_importances_)))

In [ ]:
regressor.predict([[8,4,'0030']])

In [ ]:
# def rfr_model(X, y):
#     # Perform Grid-Search
#     gsc = GridSearchCV(
#         estimator=RandomForestRegressor(),
#         param_grid={
#             'max_depth': range(3,7),
#             'n_estimators': (10, 50, 100, 1000),
#         },
#         cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
    
#     grid_result = gsc.fit(X, y)
#     best_params = grid_result.best_params_
    
#     rfr = RandomForestRegressor(max_depth=best_params["max_depth"], n_estimators=best_params["n_estimators"], random_state=False, verbose=False)
#     rfr = rfr.fit(X, y)  
    
#     # Perform K-Fold CV
# #     scores = cross_val_score(rfr, X, y, cv=10, scoring='neg_mean_absolute_error')

#     return rfr

# regressor = rfr_model(X_train, y_train) 
# y_pred = regressor.predict(X_test)